# TF-IDF介绍

In [1]:
# TF-IDF 是NLP一种常用的统计方法，即关键词的重要性随着它在一个文件中出现的次数成正比增加，但随着它在语料库中出现的频率反比增加
# tfidf = tf * idf
# tf (term frequence): 即一个词在文档中出现的频率，假设一个文档有 N个词，一个词在整个文档中出现i次，则tf = i/N
# idf (inverse document frequence): 假设一个语料库有n个文档，而一个词在k个文档中出现过，则 idf = log(n/k)


In [2]:
text1 ="""
Football is a family of team sports that involve, to varying degrees, kicking a ball to score a goal. 
Unqualified, the word football is understood to refer to whichever form of football is the most popular 
in the regional context in which the word appears. Sports commonly called football in certain places 
include association football (known as soccer in some countries); gridiron football (specifically American 
football or Canadian football); Australian rules football; rugby football (either rugby league or rugby union); 
and Gaelic football. These different variations of football are known as football codes.
"""

text2 = """
Basketball is a team sport in which two teams of five players, opposing one another on a rectangular court, 
compete with the primary objective of shooting a basketball (approximately 9.4 inches (24 cm) in diameter) 
through the defender's hoop (a basket 18 inches (46 cm) in diameter mounted 10 feet (3.048 m) high to a backboard 
at each end of the court) while preventing the opposing team from shooting through their own hoop. A field goal is 
worth two points, unless made from behind the three-point line, when it is worth three. After a foul, timed play stops 
and the player fouled or designated to shoot a technical foul is given one or more one-point free throws. The team with 
the most points at the end of the game wins, but if regulation play expires with the score tied, an additional period 
of play (overtime) is mandated.
"""

text3 = """
Volleyball, game played by two teams, usually of six players on a side, in which the players use their hands to bat a 
ball back and forth over a high net, trying to make the ball touch the court within the opponents’ playing area before 
it can be returned. To prevent this a player on the opposing team bats the ball up and toward a teammate before it touches 
the court surface—that teammate may then volley it back across the net or bat it to a third teammate who volleys it across 
the net. A team is allowed only three touches of the ball before it must be returned over the net.
"""
# 这三篇文章分别是关于足球，篮球，排球的介绍

In [3]:
# 预处理
import nltk
# nltk.download('stopwords')
import string

# 文本预处理
# 函数：text文件分句，分词，并去掉标点
def get_tokens(text):
    text = text.replace('\n', '')
    sents = nltk.sent_tokenize(text)  # 分句
    tokens = []
    for sent in sents:
        for word in nltk.word_tokenize(sent):  # 分词
            if word not in string.punctuation: # 去掉标点
                tokens.append(word)
    return tokens

In [4]:
# 去掉文章中的停用词
from nltk.corpus import stopwords     #停用词
from collections import Counter       #计数

# 对原始的text文件去掉停用词
# 生成count字典，即每个单词的出现次数
def make_count(text):
    tokens = get_tokens(text)
    filtered = [w for w in tokens if not w in stopwords.words('english')]    #去掉停用词
    count = Counter(filtered)
    return count

# Gensim 实现

In [5]:
from nltk.corpus import stopwords     #停用词
from gensim import corpora, models, matutils

# training by gensim's Ifidf Model
def get_words(text):
    tokens = get_tokens(text)
    filtered = [w for w in tokens if not w in stopwords.words('english')]
    return filtered

# get text
count1, count2, count3 = get_words(text1), get_words(text2), get_words(text3)
print("count1: ", count1)
count_list = [count1, count2, count3]
# training by TfidfModel in gensim
# count_list 就是已经完成切词、停用词
dictionary = corpora.Dictionary(count_list)
# new_dict 词典展示
new_dict = {v:k for k,v in dictionary.token2id.items()}
print("=================================================")
print("new_dict: ", new_dict)
# 针对每个text中的关键词进行编码
corpus2 = [dictionary.doc2bow(count) for count in count_list]
tfidf2 = models.TfidfModel(corpus2)
corpus_tfidf = tfidf2[corpus2]

# output
print("\nTraining by gensim Tfidf Model.......\n")
for i, doc in enumerate(corpus_tfidf):
    print("Top words in document %d"%(i + 1))
    sorted_words = sorted(doc, key=lambda x: x[1], reverse=True)    # type=list
    for num, score in sorted_words[:3]:
        print("    Word: %s, TF-IDF: %s"%(new_dict[num], round(score, 5)))


count1:  ['Football', 'family', 'team', 'sports', 'involve', 'varying', 'degrees', 'kicking', 'ball', 'score', 'goal', 'Unqualified', 'word', 'football', 'understood', 'refer', 'whichever', 'form', 'football', 'popular', 'regional', 'context', 'word', 'appears', 'Sports', 'commonly', 'called', 'football', 'certain', 'places', 'include', 'association', 'football', 'known', 'soccer', 'countries', 'gridiron', 'football', 'specifically', 'American', 'football', 'Canadian', 'football', 'Australian', 'rules', 'football', 'rugby', 'football', 'either', 'rugby', 'league', 'rugby', 'union', 'Gaelic', 'football', 'These', 'different', 'variations', 'football', 'known', 'football', 'codes']
new_dict:  {0: 'American', 1: 'Australian', 2: 'Canadian', 3: 'Football', 4: 'Gaelic', 5: 'Sports', 6: 'These', 7: 'Unqualified', 8: 'appears', 9: 'association', 10: 'ball', 11: 'called', 12: 'certain', 13: 'codes', 14: 'commonly', 15: 'context', 16: 'countries', 17: 'degrees', 18: 'different', 19: 'either', 2

# 手动实现

In [6]:
import math

# 计算tf
def tf(word, count):
    return count[word] / sum(count.values())
# 计算count_list有多少个文件包含word
def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)

# 计算idf
def idf(word, count_list):
    return math.log2(len(count_list) / (n_containing(word, count_list)))    #对数以2为底
# 计算tf-idf
def tfidf(word, count, count_list):
    return tf(word, count) * idf(word, count_list)

# 对向量做规范化, normalize
def unitvec(sorted_words):
    lst = [item[1] for item in sorted_words]
    L2Norm = math.sqrt(sum(np.array(lst)*np.array(lst)))
    unit_vector = [(item[0], item[1]/L2Norm) for item in sorted_words]
    return unit_vector


# TF-IDF测试
count1, count2, count3 = make_count(text1), make_count(text2), make_count(text3)
print("count1: ", count1)
countlist = [count1, count2, count3]
print("Training by original algorithm......\n")
for i, count in enumerate(countlist):
    print("Top words in document %d"%(i + 1))
    scores = {word: tfidf(word, count, countlist) for word in count}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)    #type=list
    sorted_words = matutils.unitvec(sorted_words)
    for word, score in sorted_words[:3]:
        print("    Word: %s, TF-IDF: %s"%(word, round(score, 5)))


count1:  Counter({'football': 12, 'rugby': 3, 'word': 2, 'known': 2, 'Football': 1, 'family': 1, 'team': 1, 'sports': 1, 'involve': 1, 'varying': 1, 'degrees': 1, 'kicking': 1, 'ball': 1, 'score': 1, 'goal': 1, 'Unqualified': 1, 'understood': 1, 'refer': 1, 'whichever': 1, 'form': 1, 'popular': 1, 'regional': 1, 'context': 1, 'appears': 1, 'Sports': 1, 'commonly': 1, 'called': 1, 'certain': 1, 'places': 1, 'include': 1, 'association': 1, 'soccer': 1, 'countries': 1, 'gridiron': 1, 'specifically': 1, 'American': 1, 'Canadian': 1, 'Australian': 1, 'rules': 1, 'either': 1, 'league': 1, 'union': 1, 'Gaelic': 1, 'These': 1, 'different': 1, 'variations': 1, 'codes': 1})
Training by original algorithm......

Top words in document 1
    Word: football, TF-IDF: 0.84766
    Word: rugby, TF-IDF: 0.21192
    Word: word, TF-IDF: 0.14128
Top words in document 2
    Word: play, TF-IDF: 0.29872
    Word: one, TF-IDF: 0.19915
    Word: shooting, TF-IDF: 0.19915
Top words in document 3
    Word: net, TF